In [ ]:
import sys
sys.path.append('../input/brown-clustering')

In [ ]:
import os
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize

from brownclustering import Corpus
from brownclustering import BrownClustering

In [ ]:
pub_df=pd.read_csv('../input/publication-datasets/publication_dataset.csv')
pub_df.head()

In [ ]:
%%time
pos_sentences=[]

def get_positive_sentences(sentences):
    sentences=eval(sentences)
    pos_sents=sentences['pos_sents']
    for sentence in pos_sents:
        sentence=sentence.lower().strip()
        words=word_tokenize(sentence)
        words=[word.strip() for word in words]
        words=[word for word in words if (word.isalpha() and word.isascii())]
        pos_sentences.append(words)

_=pub_df.sentences.apply(get_positive_sentences)
print('Number Of Positive Sentences:', len(pos_sentences))

In [ ]:
from collections import defaultdict
vocab=defaultdict(int)
for sent in pos_sentences:
    for word in sent:
        vocab[word]+=1
print('Vocab size:', len(vocab))

In [ ]:
min_freq=10
corpus_sentences=[]
for sent in pos_sentences:
    words=[word for word in sent if vocab[word]>min_freq]
    if len(words)>3:
        corpus_sentences.append(words)
print("Corpus Sentences:", len(corpus_sentences))

# Build Corpus for the sentences with brown clusters

In [ ]:
%%time
corpus=Corpus(corpus_sentences, 0.001)
vocab=corpus.vocabulary
unigrams=corpus.unigrams
bigrams=corpus.bigrams

print('Vocabulary Size:',len(vocab))
print('Unigram Size:', len(unigrams))
print("Bigram Size:", len(bigrams))

In [ ]:
clustering = BrownClustering(corpus, 6)
clustering.train()

# lets see some of the keywords for extraction

In [ ]:
clustering.get_similar('taken')

In [ ]:
clustering.get_similar('database')

In [ ]:
clustering.get_similar('collected')

In [ ]:
clustering.get_similar('sampled')

In [ ]:
clustering.get_similar('from')

In [ ]:
clustering.get_similar('this')

# Create Clusters

In [ ]:
codes=clustering.codes()
print(len(codes))

creating codes of length 5

In [ ]:
codes_df=pd.DataFrame.from_dict({
    'word': list(codes.keys()),
    'code_':list(codes.values())
})
codes_df.head()

In [ ]:
clustering.get_similar('obtained', 30)

Lets check the clusters of the context words of database label.

In [ ]:
prefix=10

text1="the data is taken from ADNI"
text2="This study used data from the National Education Longitudinal Study (NELS:88) to examine the effects of dual enrollment programs for high school students on college degree attainment."

print(text1)
print()
for word in word_tokenize(text1):
    word=word.lower()
    if word not in codes:
        continue
    print(codes[word][:prefix], "-->", word)

print(text2)
print()
for word in word_tokenize(text2):
    word=word.lower()
    if word not in codes:
        continue
    print(codes[word][:prefix], "-->", word)

In [ ]:
codes_df[codes_df.code_.apply(lambda x: x.startswith('0100000000'))]